In [52]:
import pandas as pd
import os

In [60]:
path = "/Users/danielcm/Desktop/Sycuro/Projects/Diabetes/t1d_db_fixed_discussed/"
os.chdir(path)

In [76]:
#This section makes a new file (and df) for every barrnap blast output with the necessary headers and adds the number of genus, and species found for every sample individually.
#Then, it filters by 98.6 identity or higher, and by e-val of 1e-100
master_file = pd.read_csv("FINAL_report_WGS_all_csv.csv")
subset_master_file = master_file[['Index','Sample ID','Consortia','Selected for Downstream',
                    'Sanger classification (v3v4) Source','Classification species']]
input_file_path = path+"barrnap_blast/"
subset_master_file.set_index('Sample ID',inplace=True)
#Manually removed 073 and 100, as they did not have any output in the file except
sample_and_species_dict = {}
for file in sorted(os.listdir(input_file_path)):
    blast_file = pd.read_csv(input_file_path+file,header=None)
    blast_file.columns = ["Sequence ID","Matched ID","Bit score","Score","E-value","% of identity","Sequence"]
    number_of_16S = len(blast_file['Sequence ID'].unique())
    blast_file['Number of 16S found'] = number_of_16S
    filtered_blast_file = blast_file[(blast_file['% of identity']>=98.6) & (blast_file['E-value']<=1e-100)].copy() #Filtering by e-value and identity
    blast_species = filtered_blast_file['Matched ID'].tolist()
    matched_species = []
    for item in blast_species:
        split = item.split(" ",1)
        species_name = split[-1]
        matched_species.append(species_name)
    filtered_blast_file['Matched species'] = matched_species #All of them that were found at high quality
    tmp_id_name = "_".join(file.split("_")[3:7]).split(".",1)[0] #This assigns just the sample ID without the .csv or the 'prokka_blast' part
    filtered_blast_file["Sample ID"] = tmp_id_name
    unique_species = filtered_blast_file["Matched species"].unique() #This makes an array (or a list?) of unique species found by BLASTn
    genus_list = []
    for item in blast_species:
        genus = item.split(" ")[1]
        genus_list.append(genus)
    sample_and_species_dict[tmp_id_name] = unique_species.tolist()
    filtered_blast_file['Matched genus'] = genus_list
    unique_genus = filtered_blast_file["Matched genus"].unique() #This makes a list of unique genus found by BLASTn
    unique_species = list(unique_species)
    unique_genus = list(unique_genus)
    filtered_blast_file['Unique species'] = [unique_species]*len(filtered_blast_file)
    filtered_blast_file['Unique genus'] = [unique_genus]*len(filtered_blast_file)
    filtered_blast_file['No. of unique species'] = len(unique_species)
    filtered_blast_file['No. of unique genera'] = len(unique_genus)
    filtered_blast_file = filtered_blast_file[['Sample ID','Sequence ID','Matched ID','Bit score','Score','E-value','% of identity','Number of 16S found',
                          'Matched species','No. of unique species','Unique species','Matched genus','No. of unique genera','Unique genus','Sequence']]
    
    filtered_blast_file.to_csv(tmp_id_name+"blast_filtered.csv",sep=',') #Sample id as output

In [78]:
#Confirming match between WGS, 16S v3v4 and BLAST
#This script matches the 16S found in the genomes and compares it againts the output of gtdbtk and sanger (v3v4) from Toronto
#if it matches gtdbtk, then it is used. If it does not matches gtdbtk, but matches v3v4, then it is also used. Priority is given to 
#gtdbtk
comparison_list=[]
fasta_out = open("fasta_16S_in_silico.fasta","w")
for sample_id,unique_species in sample_and_species_dict.items():
    gtdbtk_species = subset_master_file.loc[sample_id,'Classification species']
    sanger_species = subset_master_file.loc[sample_id,'Sanger classification (v3v4) Source']
    match1 = gtdbtk_species in unique_species
    match2 = sanger_species in unique_species
    file_name = sample_id+"blast_filtered.csv"
    file_to_retrive_sequence = pd.read_csv(path+file_name)
    matching_sequence = file_to_retrive_sequence[file_to_retrive_sequence['Matched species'] == gtdbtk_species]['Sequence'].tolist()
    if not matching_sequence:
        matching_sequence = file_to_retrive_sequence[file_to_retrive_sequence['Matched species']==sanger_species]['Sequence'].tolist()
    comparison_list.append({
        'Sample ID':sample_id,
        'GTDBtk species':gtdbtk_species,
        'Sanger species':sanger_species,
        'Match blast & gtdbtk?':match1,
        'Match blast & sanger?':match2,
        'Sequences':matching_sequence
    })
    for sequence in matching_sequence:
        fasta_out.write(">"+sample_id+"\n")
        fasta_out.write(sequence+"\n")
df_final = pd.DataFrame(comparison_list)
sequences_expanded = df_final['Sequences'].apply(pd.Series)
df_final = df_final.drop('Sequences', axis=1).join(sequences_expanded)
df_final.to_csv("Final_16S_sequences_from_genomes.csv")
fasta_out.close()

/var/folders/tm/fnbprp5d6gb3vjll_w5r4k3m0000gn/T/ipykernel_83151/97078124.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sequences_expanded = df_final['Sequences'].apply(pd.Series)


In [ ]:
#This section creates the fasta file of the matched sequences